# Check basic statistics of manually selected topics
**Objective**: make sure manually selected topics have high quality.  
 - Characteristic keywords: easy to regonize associated news.
 - Amout of disscussion: reasonable size of associated news and tweets
 - Consistent in meaning: no drift/disperse in content.
 - Evolution of event: reasonable time-span of associated news.
 
Last modified: 2017-10-18

# Roadmap
1. Manually compile a list of topics with keywords
2. Check number of associated news and tweets for each topic
3. Check news titles and sample tweets of each topic
4. Check time-span of each topic

# Steps

In [1]:
"""
Initialization
"""

'''
Standard modules
'''
import os
import pickle
import sqlite3
import time
from pprint import pprint

'''
Analysis modules
'''
import pandas as pd


'''
Custom modules
'''
import config
import utilities

'''
Misc
'''
nb_name = '20171011-daheng-check_topics_basic_statistics'

## Manually compile a list of topics with keywords
Topics information (category, name, keywords_lst) are manually compiled into config.MANUALLY_SELECTED_TOPICS_LST

In [2]:
"""
Print out manually selected topics information
"""
for topic_ind, topic in enumerate(config.MANUALLY_SELECTED_TOPICS_LST):
    print('({}/{}) {}'.format(topic_ind+1, len(config.MANUALLY_SELECTED_TOPICS_LST), topic))

(1/51) {'category': 'politics', 'name': 'Hillary_Clinton_email_controversy', 'keywords_lst': [('email', 'e-mail'), ('Hillary', 'Clinton')]}
(2/51) {'category': 'politics', 'name': 'Iran_nuclear_deal', 'keywords_lst': ['Iran', 'nuclear']}
(3/51) {'category': 'politics', 'name': 'ISIS_Jihadi_John_identity_reveal', 'keywords_lst': ['Jihadi John']}
(4/51) {'category': 'politics', 'name': 'Ukraine_cease_fire', 'keywords_lst': [('cease-fire', 'ceasefire'), ('Ukraine', 'Russia')]}
(5/51) {'category': 'politics', 'name': 'Egypt_free_Al_Jazeera_journalist', 'keywords_lst': [('Al Jazeera', 'Egypt'), ('Peter Greste', 'journalist')]}
(6/51) {'category': 'politics', 'name': 'Keystone_XL_Pipeline_bill', 'keywords_lst': ['Keystone XL']}
(7/51) {'category': 'politics', 'name': 'CIA_Torture_Report', 'keywords_lst': ['Torture Report']}
(8/51) {'category': 'politics', 'name': 'Obama_cybersecurity_plan', 'keywords_lst': ['Obama', 'cyber']}
(9/51) {'category': 'politics', 'name': 'DHS_funding_issue', 'keyw

## Check number of associated news and tweets for each topic

### Build pickle for news_id and tweets_id associated with each topic

In [3]:
%%time
"""
Register
    TOPICS_LST_PKL = os.path.join(DATA_DIR, 'topics.lst.pkl')
in config.
"""
if 0 == 1:
    supplement_topics_lst = []
    
    '''
    Load in pickle for news data over selected period.
    '''
    news_period_df = pd.read_pickle(config.NEWS_PERIOD_DF_PKL)
 
    for topic_ind, topic in enumerate(config.MANUALLY_SELECTED_TOPICS_LST):
        localtime = time.asctime(time.localtime(time.time()))
        print('({}/{}) processing topic: {} ... {}'.format(topic_ind+1,
                                                           len(config.MANUALLY_SELECTED_TOPICS_LST),
                                                           topic['name'],
                                                           localtime))
        '''
        Match out associated news titles.
        '''
        asso_news_native_ids_lst = []
        for ind, row in news_period_df.iterrows():
            if utilities.news_title_match(row['news_title'], topic['keywords_lst'], verbose=False):
                asso_news_native_ids_lst.append(row['news_native_id'])
        
        topic['news_native_ids_lst'] = asso_news_native_ids_lst
        
        '''
        Query associated tweets
        '''
        asso_tweets_ids_lst = []
        
        query_news_tweets = '''
        select tweet_id from tweets
        where news_native_id = :news_native_id
        order by tweet_id asc;'''
        
        with sqlite3.connect(config.NEWS_TWEETS_DB_FILE) as conn:
            cursor = conn.cursor()
            for news_native_id in topic['news_native_ids_lst']:
                cursor.execute(query_news_tweets, {'news_native_id': news_native_id})
                tweets_ids_lst = [item[0] for item in cursor.fetchall()]
                asso_tweets_ids_lst.extend(tweets_ids_lst)
                
        topic['tweets_ids_lst'] = asso_tweets_ids_lst
        
        supplement_topics_lst.append(topic)
    
    '''
    Make pickle
    '''
    with open(config.TOPICS_LST_PKL, 'wb') as f:
        pickle.dump(supplement_topics_lst, f)

(1/51) processing topic: Hillary_Clinton_email_controversy ... Wed Oct 18 17:01:55 2017
(2/51) processing topic: Iran_nuclear_deal ... Wed Oct 18 17:02:03 2017
(3/51) processing topic: ISIS_Jihadi_John_identity_reveal ... Wed Oct 18 17:02:14 2017
(4/51) processing topic: Ukraine_cease_fire ... Wed Oct 18 17:02:22 2017
(5/51) processing topic: Egypt_free_Al_Jazeera_journalist ... Wed Oct 18 17:02:30 2017
(6/51) processing topic: Keystone_XL_Pipeline_bill ... Wed Oct 18 17:02:38 2017
(7/51) processing topic: CIA_Torture_Report ... Wed Oct 18 17:02:45 2017
(8/51) processing topic: Obama_cybersecurity_plan ... Wed Oct 18 17:02:53 2017
(9/51) processing topic: DHS_funding_issue ... Wed Oct 18 17:03:01 2017
(10/51) processing topic: US_Cuba_relationship ... Wed Oct 18 17:03:08 2017
(11/51) processing topic: 2015_CPAC ... Wed Oct 18 17:03:17 2017
(12/51) processing topic: Iraq_free_ISIS_Tikrit ... Wed Oct 18 17:03:25 2017
(13/51) processing topic: Nigeria_Boko_Haram_terrorists ... Wed Oct 18 

### Recover pickle and print number of news and tweets for each topic

In [4]:
"""
Test recover topics lst pkl
"""
if 0 == 1:
    with open(config.TOPICS_LST_PKL, 'rb') as f:
        topics_lst = pickle.load(f)
    
    for topic_ind, topic in enumerate(topics_lst):
        print('{} Topic_name: {}; news_num: {}; tweets_num: {}'.format(topic_ind,
                                                                       topic['name'],
                                                                       len(topic['news_native_ids_lst']),
                                                                       len(topic['tweets_ids_lst'])))

0 Topic_name: Hillary_Clinton_email_controversy; news_num: 228; tweets_num: 860564
1 Topic_name: Iran_nuclear_deal; news_num: 406; tweets_num: 2412264
2 Topic_name: ISIS_Jihadi_John_identity_reveal; news_num: 101; tweets_num: 620121
3 Topic_name: Ukraine_cease_fire; news_num: 84; tweets_num: 603709
4 Topic_name: Egypt_free_Al_Jazeera_journalist; news_num: 50; tweets_num: 129120
5 Topic_name: Keystone_XL_Pipeline_bill; news_num: 55; tweets_num: 117692
6 Topic_name: CIA_Torture_Report; news_num: 41; tweets_num: 167362
7 Topic_name: Obama_cybersecurity_plan; news_num: 73; tweets_num: 495576
8 Topic_name: DHS_funding_issue; news_num: 45; tweets_num: 104911
9 Topic_name: US_Cuba_relationship; news_num: 235; tweets_num: 1213314
10 Topic_name: 2015_CPAC; news_num: 68; tweets_num: 289774
11 Topic_name: Iraq_free_ISIS_Tikrit; news_num: 94; tweets_num: 567544
12 Topic_name: Nigeria_Boko_Haram_terrorists; news_num: 243; tweets_num: 954810
13 Topic_name: Ferguson_unrest; news_num: 611; tweets_num:

## Check news titles and sample tweets of each topic

In [5]:
"""
Select a topic, print out name of topic
"""
if 1 == 1:
    with open(config.TOPICS_LST_PKL, 'rb') as f:
        topics_lst = pickle.load(f)
    
    topic = topics_lst[1]

In [6]:
'''
Print associated news titles
'''
if 1 == 1:
    print('TOPIC: {}; KEYWORDS: {}'.format(topic['name'], topic['keywords_lst']))
    
    # limit to first 100 news
    news_native_ids_lst = topic['news_native_ids_lst'][:100]
    
    query_news = '''
    select news_title, news_collected_time from news
    where news_native_id = :news_native_id
    order by news_native_id asc;'''

    with sqlite3.connect(config.NEWS_TWEETS_DB_FILE) as conn:
        conn.row_factory = sqlite3.Row
        cursor = conn.cursor()
        for news_native_id in news_native_ids_lst:
            cursor.execute(query_news, {'news_native_id': news_native_id})
            for row in cursor.fetchall():
                print('{}: {}'.format(row['news_collected_time'], row['news_title']))

TOPIC: Iran_nuclear_deal; KEYWORDS: ['Iran', 'nuclear']
2014-11-19: Britain 'not optimistic' on Iran nuclear talks
2014-11-20: Kerry to Join Iran Nuclear Talks in Vienna
2014-11-20: Kerry to travel to Vienna for Iran nuclear talks
2014-11-20: Iran not providing explanations on nuclear claims: watchdog
2014-11-20: For many Iranians, nuclear talks aren't about the sanctions
2014-11-20: Iranians Count Sanctions Cost as Nuclear Accord Deadline Nears
2014-11-20: Iran nuclear talks deadline may be extended to March: officials
2014-11-20: Iran nuclear talks stuck, deadline may be extended officials
2014-11-20: Kerry: Iran nuclear talks seek agreements not extension in upcoming deadline
2014-11-20: Kerry Meets French Counterpart Ahead of Iran Nuclear Talks in Vienna
2014-11-20: Iran nuclear talks stuck, deadline may be extended officials
2014-11-20: Iran nuclear deal? Maybe with a small "d"
2014-11-20: John Kerry Heads to Vienna for Iran Nuclear Talks
2014-11-20: Middle East|US Lays Out Limits

In [7]:
'''
Print associated tweets
'''
if 1 == 1:
    print('TOPIC: {}; KEYWORDS: {}'.format(topic['name'], topic['keywords_lst']))
    
    # limit to first 100 tweets
    tweets_native_ids_lst = topic['tweets_native_ids_lst'][:100]
    
    query_tweets = '''
    select tweet_text, tweet_collected_time from tweets
    where tweet_native_id = :tweet_native_id
    order by tweet_native_id asc;'''

    with sqlite3.connect(config.NEWS_TWEETS_DB_FILE) as conn:
        conn.row_factory = sqlite3.Row
        cursor = conn.cursor()
        for tweet_native_id in tweets_native_ids_lst:
            cursor.execute(query_tweets, {'tweet_native_id': tweet_native_id})
            for row in cursor.fetchall():
                print('{}: {}'.format(row['tweet_collected_time'], row['tweet_text']))

TOPIC: Iran_nuclear_deal; KEYWORDS: ['Iran', 'nuclear']
2014-11-19: Us congress,JudgeTedPoe to convene hrg entitled Iranian Nuclear Talks: Negotiating a Bad Deal? http://t.co/WPkUx1jGLk  #Iran #NoNuclearIran
2014-11-19: #Iran U.S. Nov. 24 Nuclear Deal:  Love Me, Love me not, Love Meeeeeeee http://t.co/sz4BubPbCs http://t.co/RQ95bU3xx1
2014-11-19: #Event Today at2PM ET US. #Congress suspicion about Iran Nuke deal!  #IranTalksVienna http://t.co/WPkUx1jGLk  #Iran #NoNuclearIran
2014-11-19: U.K. Official Says Deadline for Iran Talks Unlikely to Be Met (WSJ) http://t.co/LEeTKDHSRW
2014-11-19: State of Play as Iran Talks Enter Final Week - Early Warning http://t.co/pMwIgLjKSM
2014-11-19: Talks between US and Iran haunted by difficult history http://t.co/VXbr4i3O6w
2014-11-19: Doubts about a nuclear deal with #Iran as talks near end #irantalks #IranTalksvienna http://t.co/z37eZmfwiB
2014-11-19: #UN Rebukes #Iran on Human Rights as #Nuclear Talks Near Deadline http://t.co/IxeWoD1L08
2014-11-19